Train Model

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


In [6]:
# Set the image size and batch size
img_size = (255, 255)
batch_size = 5

# Define transforms for the training and validation sets
transform = transforms.Compose([
    transforms.Resize((img_size[0], img_size[1])),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define the data loaders
train_dir = '../Data/Train/'
val_dir = '../Data/Test/'

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)


In [7]:
print(train_dataset.classes)


['Green_Bad', 'Green_Good', 'Yellow_Bad', 'Yellow_Good']


In [6]:
import torch.nn as nn


class MangoModel(nn.Module):
    def __init__(self):
        super(MangoModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32*63*63, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(64, 4)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [9]:
def train(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss


def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(val_loader.dataset)
    epoch_acc = correct_preds.double() / len(val_loader.dataset)
    return epoch_loss, epoch_acc


model = MangoModel().to(device)


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
best_val_loss = float('inf')
count = 1

for epoch in range(1):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    print(
        f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model_' + str(count) + '.pth')
        count = count + 1


Predict

In [8]:
import torch

# Load the model
class_names = ['Green_Bad', 'Green_Good', 'Yellow_Bad', 'Yellow_Good']

model = MangoModel()
model.load_state_dict(torch.load('../Save/Mango_Pytorch_98_0.14.pth', map_location=torch.device('cpu')))
model.eval()


MangoModel(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=127008, out_features=64, bias=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=64, out_features=4, bias=True)
)

In [9]:
import torchvision.transforms as transforms
from PIL import Image

# Define a transform to preprocess the image
transform = transforms.Compose([
    transforms.Resize((255, 255)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


def load_image(image_path):
    # Load the image using PIL
    image = Image.open(image_path)
    # Preprocess the image using the defined transform
    preprocessed_image = transform(image)
    # Add a batch dimension to the preprocessed image
    preprocessed_image = preprocessed_image.unsqueeze(0)
    # Pass the preprocessed image through the model

    return preprocessed_image


In [13]:
# Load and transform image
image_path = '../Data/Test/Yellow_Good/47.jpg'
preprocessed_image = load_image(image_path)

# Predict image
output = model.forward(preprocessed_image)
# Get the probabilities for all classes
probabilities = torch.nn.functional.softmax(output, dim=1)

# Print the probabilities for all classes
print("Class probabilities:")
for i, prob in enumerate(probabilities.squeeze()):
    print("Class {}: {} ({:.2f}%)".format(i, class_names[i], prob*100))


Class probabilities:
Class 0: Green_Bad (0.11%)
Class 1: Green_Good (0.45%)
Class 2: Yellow_Bad (0.12%)
Class 3: Yellow_Good (99.32%)


In [11]:
image_paths = ['../Data/Test/Green_Good/47.jpg',
               '../Data/Test/Green_Good/49.jpg']

probabilities_sum = None
for image_path in image_paths:
    preprocessed_image = load_image(image_path)
    output = model.forward(preprocessed_image)
    probabilities = torch.nn.functional.softmax(output, dim=1)

    if probabilities_sum is None:
        probabilities_sum = probabilities
    else:
        probabilities_sum += probabilities

average_probabilities = probabilities_sum / len(image_paths)
for i, prob in enumerate(average_probabilities.squeeze()):
    print("Class {}: {} ({:.2f}%)".format(i, class_names[i], (prob)*100))


Class 0: Green_Bad (0.36%)
Class 1: Green_Good (99.45%)
Class 2: Yellow_Bad (0.11%)
Class 3: Yellow_Good (0.08%)
